# Análise de dados do dataset Titanic
---

Aqui, será necessário explorar os dados com um olhar mais crítico, identificar padrões  e aplicar técnicas que vão além do básico.

Essa base de dados não exige muita preparação, mas será necessário tratar valores ausentes:

Preencher os valores ausentes da coluna Age com a mediana por grupo (Sex + Pclass)  
Obs.: temos valores decimais na coluna Age, esse valores são estimativas ou registros mais precisos (às vezes idade de crianças, por exemplo). Portanto, não precisa arredondar ou excluir por conta disso.

In [31]:
import pandas as pd
import numpy as np
import re
import warnings

# Ignorar warnings futuras
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
df = pd.read_csv('Titanic-Dataset.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


* ``Survived`` - Sobreviveu (0 = não; 1 = sim)
* ``Pclass`` - Classe do passageiro (1 = primeira classe; 2 = segunda classe; 3 = terceira classe)
* ``Name`` - Nome
* ``Sex`` - Sexo
* ``Age`` - Idade
* ``SibSp`` - Número de irmãos/cônjuges a bordo
* ``Parch`` - Número de pais/crianças a bordo
* ``Ticket`` - Número do bilhete
* ``Fare`` - Tarifa do Passageiro
* ``Cabin`` - Cabine
* ``Embarked`` - Porto de Embarque (C = Cherbourg; Q = Queenstown; S = Southampton)

In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

### Tipagem dos dados

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### Resumo estatístico das colunas

In [6]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


### Vamos contar os valores nulos para cada coluna

In [7]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### Vamos substituir os valores ausentes em Age pela mediana por grupo (Sex + Pclass)

Para isso, utilizaremos o método **`.transform()`** do pandas. É um método que é usado após um **``.groupby()``** e retorna uma série com o mesmo índice que o DataFrame original. 

Isso é importante porque significa que os dados "encaixam" de volta no DataFrame, linha por linha.

In [8]:
medianas = df.groupby(['Sex', 'Pclass'])['Age'].transform('median')
medianas.isnull().sum()

np.int64(0)

**``medianas``** é uma série com o **mesmo tamanho de df**, onde cada valor corresponde à mediana de ``Age`` dentro do grupo (``Sex``, ``Pclass``) daquela linha. O número de medianas únicas depende do número de grupos distintos existentes no DataFrame.
* medianas é uma Série com o mesmo número de linhas que df.
* Cada linha recebe a mediana do grupo correspondente (Sex + Pclass).
* O número de grupos únicos com uma mediana é igual ao número de combinações distintas de Sex e Pclass no DataFrame.

In [9]:
# Total de combinações
df[['Sex', 'Pclass']].drop_duplicates().shape[0]

6

### Vamos ver o que acontece se aplicarmos `medianas` no .fillna(). Quantos valos nulos teremos?

In [10]:
df['Age'].fillna(medianas).isnull().sum()

np.int64(0)

In [11]:
df['Age'] = df['Age'].fillna(medianas)
df.isnull().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Observe que todos os valores nulos de ``Age`` sumiram e agora assumem a mediana dos seus respectivos grupos.

In [12]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,21.5,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


### Vamos à análise:

## 1. Quantos passageiros embarcaram em S e estavam viajando com pelo menos 3 familiares?  

In [30]:
# Quantos passageiros embarcaram em S
# O operador & tem precedência maior que == e >=
df[(df['Embarked']=='S') & (df['SibSp'] >= 3)].shape[0]

42

## 2. Qual é o número total de passageiros com o sobrenome Smith? 
(Exatamente Smith, não considerar variações como blacksmith)

In [33]:
# Utilizaremos de expressões regulares para solucionar esse problema

def separar_nome(nome):
    padrao = r'^([^,]+),\s([^\.]+)\.\s([^"]+)(?:\s"([^"]+)")?'
    match = re.match(padrao, nome)
    if match:
        return match.groups()
    return (None, None, None, None)

df[['Sobrenome', 'Titulo', 'Nome_Principal', 'Apelido']] = df['Name'].apply(separar_nome).apply(pd.Series)

In [37]:
df[df['Sobrenome'] == 'Smith']

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sobrenome,Titulo,Nome_Principal,Apelido
174,175,0,1,"Smith, Mr. James Clinch",male,56.0,0,0,17764,30.6958,A7,C,Smith,Mr,James Clinch,None
260,261,0,3,"Smith, Mr. Thomas",male,25.0,0,0,384461,7.7500,NaN,Q,Smith,Mr,Thomas,None
284,285,0,1,"Smith, Mr. Richard William",male,40.0,0,0,113056,26.0000,A19,S,Smith,Mr,Richard William,None
346,347,1,2,"Smith, Miss. Marion Elsie",female,40.0,0,0,31418,13.0000,NaN,S,Smith,Miss,Marion Elsie,None


4 pessoas possuem o sobrenome Smith.

## 3. Qual a mediana das idade das mulheres da 1ª Classe?
Se a resposta for um número decimal, utilize o ponto como separador decimal e não use separadores para os milhares.

In [43]:
print(df[(df['Sex']=='female') & (df['Pclass']== 1 )]['Age'].median())

35.0


## 4. Entre a 1ª e a 3ª classe, a variação percentual na taxa de mortalidade foi maior entre mulheres do que entre homens?

In [111]:
# Taxa de mortalidade de homens na primeira classe
htx1 = df[(df['Pclass']==1) & (df['Sex']=='male') & (df['Survived']==0)].shape[0]/df[(df['Pclass']==1) & (df['Sex']=='male')].shape[0]
print(htx1)

0.6311475409836066


In [110]:
# Taxa de mortalidade de homens na terceira classe
htx3 = df[(df['Pclass']==3) & (df['Sex']=='male') & (df['Survived']==0)].shape[0]/df[(df['Pclass']==3) & (df['Sex']=='male')].shape[0]
print(htx3)

0.8645533141210374


In [112]:
print('A variação da taxa de mortalidade entre 3 classe e 1 classe, para homens é de:', 100*(htx3-htx1))

A variação da taxa de mortalidade entre 3 classe e 1 classe, para homens é de: 23.340577313743083


In [ ]:
# Taxa de mortalidade em mulheres da primeira classe
ftx1 = df[(df['Pclass']==1) & (df['Sex']=='female') & (df['Survived']==0)].shape[0]/df[(df['Pclass']==1) & (df['Sex']=='female')].shape[0]
print(ftx1)

0.031914893617021274


In [114]:
# Taxa de mortalidade em mulheres da terceira classe
ftx3 = df[(df['Pclass']==3) & (df['Sex']=='female') & (df['Survived']==0)].shape[0]/df[(df['Pclass']==3) & (df['Sex']=='female')].shape[0]
print(ftx3)

0.5


In [115]:
print('A variação da taxa de mortalidade entre 3 classe e 1 classe, para mulheres é de:', 100*(ftx3-ftx1))

A variação da taxa de mortalidade entre 3 classe e 1 classe, para mulheres é de: 46.808510638297875


A maior variação se deu entre as mulheres da primeira classe a terceira, num total de aproximadamente 47 pontos percentuais.